In [1]:
import numpy as np
import tensorflow as tf
from keras.models import Sequential
from keras.layers import Dense, BatchNormalization, Dropout, LeakyReLU, Input
from scikeras.wrappers import KerasClassifier
from sklearn.model_selection import RandomizedSearchCV, StratifiedKFold
from scikeras.wrappers import KerasClassifier
from sklearn.base import ClassifierMixin
from sklearn.datasets import make_classification
from sklearn.model_selection import train_test_split

def create_custom_mlp(hidden_layer_configs=(50, 50),
                        dropout_rate=0.3,
                        activation='leaky_relu',
                        learning_rate=0.001,
                        meta=None):
    if meta is None:
        raise ValueError("meta parametresi eksik")
        
    n_features_in = meta["n_features_in_"]
    layer1_neurons, layer2_neurons = hidden_layer_configs
    model = Sequential(name="custom_MLP_trial")
    model.add(Input(shape=(n_features_in,)))
    
    model.add(Dense(layer1_neurons))
    model.add(BatchNormalization())
    if activation == 'leaky_relu': model.add(LeakyReLU(alpha=0.1))
    else: model.add(tf.keras.layers.ReLU())
    model.add(Dropout(dropout_rate))
    
    if layer2_neurons is not None and layer2_neurons > 0:
        model.add(Dense(layer2_neurons))
        model.add(BatchNormalization())
        if activation == 'leaky_relu':
            model.add(LeakyReLU(alpha=0.1))
        else:
            model.add(tf.keras.layers.ReLU())
        model.add(Dropout(dropout_rate))
    model.add(Dense(1, activation='sigmoid'))
    
    return model

X, y = make_classification(
    n_samples=500, n_features=100, n_informative=50,
    n_redundant=20, n_classes=2, random_state=42
)

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)
print(f"Veri seti oluşturuldu. Eğitim seti boyutu: {X_train.shape}, Test seti boyutu: {X_test.shape}")

keras_estimator = KerasClassifier(
    model=create_custom_mlp,
    loss='binary_crossentropy',
    optimizer=tf.keras.optimizers.Adam,
    optimizer__learning_rate=0.001,
    epochs=20,
    batch_size=32,
    verbose=0,
    callbacks=[tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=5, verbose=0, restore_best_weights=True)]
)

keras_estimator._estimator_type = "classifier"

param_distributions = {
    'model__hidden_layer_configs': [(64, 32), (128, 64), (100, None)],
    'model__dropout_rate': [0.2, 0.4],
    'model__activation': ['relu', 'leaky_relu'],
    'optimizer__learning_rate': [0.001, 0.005]
}
    
cv_strategy = StratifiedKFold(n_splits=3, shuffle=True, random_state=42)
search_cv = RandomizedSearchCV(
    estimator=keras_estimator,
    param_distributions=param_distributions,
    n_iter=4,
    cv=cv_strategy,
    scoring='f1',
    verbose=2,
    random_state=42
)
try:
    fit_params = {'validation_split': 0.2}
    search_cv.fit(X_train, y_train, **fit_params)
    print(f"En iyi F1 skoru (CV): {search_cv.best_score_:.4f}")
    print("En iyi parametreler:")
    print(search_cv.best_params_)
    best_model = search_cv.best_estimator_
    test_score = best_model.score(X_test, y_test)
    print(f"\naccuracy: {test_score:.4f}")
        
except Exception as e:
        print(f"\nBir hata oluştu: {e}")
        import traceback
        traceback.print_exc()

Veri seti oluşturuldu. Eğitim seti boyutu: (400, 100), Test seti boyutu: (100, 100)

Bir hata oluştu: 'super' object has no attribute '__sklearn_tags__'


Traceback (most recent call last):
  File "C:\Users\BerenÜnveren\AppData\Local\Temp\ipykernel_20968\3128233426.py", line 83, in <module>
    search_cv.fit(X_train, y_train, **fit_params)
  File "C:\Users\BerenÜnveren\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\sklearn\base.py", line 1389, in wrapper
    return fit_method(estimator, *args, **kwargs)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "C:\Users\BerenÜnveren\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\sklearn\model_selection\_search.py", line 933, in fit
    cv_orig = check_cv(self.cv, y, classifier=is_classifier(estimator))
                                              ^^^^^^^^^^^^^^^^^^^^^^^^
  File "C:\Users\BerenÜnveren\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\sklearn\base.p